Задание 1. (5 баллов) 
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf



In [8]:
import nltk
import re
from collections import Counter, defaultdict
import numpy as np
import copy
from gensim.models.phrases import Phrases

In [9]:
with open('stranger.txt', encoding='utf-8') as file: #Robert Heinlein's Stranger in a Strange Land with preface removed
    stranger = file.read()

In [10]:
stranger = stranger.replace('“Stranger In A Strange Land” by Robert Heinlein', '')

In [11]:
sents = nltk.tokenize.sent_tokenize(stranger)

In [12]:
pattern = re.compile(r'([A-Za-z]+[\'-]?[A-Za-z]*)')

In [13]:
sents = [re.findall(pattern, sent) for sent in sents]

In [14]:
tri_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in sents:
    for w1, w2, w3 in nltk.trigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        tri_model[(w1, w2)][w3] += 1

Логарифм не использую, потому что верятности нужны только, чтобы сделать взвешенную выборку, а не чтобы их глазами сравнивать.

In [15]:
for bigram in tri_model:
    total_count = sum(tri_model[bigram].values())
    for target in tri_model[bigram]:
        tri_model[bigram][target] /= total_count

In [16]:
def tri_generate(model, start=('<s>', '<s>')):
    text = list(start)
    while text[-1] != '</s>': 
        index = tuple(text[-2:])
        keys = list(model[index].keys())
        values = list(model[index].values())
        key = np.random.choice(keys, 1, values)[0]
        text.append(key)
    return ' '.join(text[2:]).strip(' </s>')

In [17]:
def text_generator(sent_generator, model, number_of_sents=1, count_words=False):
    result = []
    for _ in range(number_of_sents):
        result.append(sent_generator(model))
       
    if count_words == True:
        count = count_words_avg(result)
        return count
    else:
        result = '. '.join(result) + '.'
        return result


In [18]:
def count_words_avg(sents):
    total = 0
    for sent in sents:
        total += len(sent.split(' '))
    return total/len(sents)

In [23]:
tri_example = text_generator(tri_generate, tri_model, 6)
tri_example

'Other men were in strange surrounding. Eight humans crowded together like monkeys for almost three thousand years or Terran year. Double brat Sir you are about that nominal colony we left Earth. Water on the day in debauchery. Strike that from the state prison just Out of it is practically a null area under the supervision of a troublesome question. Shoot.'

In [24]:
bi_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in sents:
    for w1, w2 in nltk.bigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        bi_model[w1][w2] += 1

In [25]:
for unigram in bi_model:
    total_count = sum(bi_model[unigram].values())
    for target in bi_model[unigram]:
        bi_model[unigram][target] /= total_count

In [26]:
def bi_generate(model, start=['<s>']):
    text = copy.deepcopy(start)
    while text[-1] != '</s>': 
        index = text[-1]
        keys = list(model[index].keys())
        values = list(model[index].values())
        key = np.random.choice(keys, 1, values)[0]
        text.append(key)
    return ' '.join(text[1:]).strip(' </s>')

In [27]:
bi_example = text_generator(bi_generate, bi_model, 6)
bi_example

"Dorcas-upstairs the victim. not see until at you darlings-and me I'll keep back out God's elect. Spoilsport. Pour me add some pretty pictures only to disturb a dog to fill with. Hans Christian Scientists indeed ready cooperation in welding strong to many a can repeat that claptrap-a written in store for example- they inclined to distribute the entrance on Mars-and hypoxia can judge them taken in devising bigger. Quite a tribal shamans will make better put this first Jill Don't tell Mike sliced across from him arrange to shut off to glimpse Agnew go regulation on Are they need an inconspicuous travel together neither Foster was sobbing and here's pay as levitation you strength-physical strength to at Pat's got up two I ordinarily do without attempting to Miriam tapped him dead issue."

Результат триграммной модели для сравнения: <br> <br>

In [28]:
tri_example

'Other men were in strange surrounding. Eight humans crowded together like monkeys for almost three thousand years or Terran year. Double brat Sir you are about that nominal colony we left Earth. Water on the day in debauchery. Strike that from the state prison just Out of it is practically a null area under the supervision of a troublesome question. Shoot.'

Можно отметить две вещи: <br>
1) Предложения, созданные триграммной модели определённо больше похожи на текст, написанный человеком. Они более связные (только грамматически, естественно). <br>
2) Биграммные предложения длиннее, чем триграммные, при условии, что мы останавливаемся только на символе окончания предложения. Это подтверждается экспериментом ниже.

In [39]:
n = 100
m = 10
bi_test = 0
tri_test = 0
for _ in range(n):
    bi_test += text_generator(bi_generate, bi_model, m, count_words=True)
    tri_test += text_generator(tri_generate, tri_model, m, count_words=True)
print(f'Average bigram model sentence length: {bi_test/n:.2f} words\n' +
      f'Average trigram model sentence length: {tri_test/n:.2f} words\n')

Average bigram model sentence length: 18.84 words
Average trigram model sentence length: 11.91 words



Задание 2. (5 баллов) 
При помощи gensim.models.Phrases реализуйте byte-pair-encoding, про который говорилось на первом семинаре (https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Preprocessing.ipynb) 
А именно 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на отдельные символы (не потеряйте пробелы) 2) обучите gensim.models.Phrases на полученных символьных предложениях 3) примените полученный нграммер к этим символьным предложениям 4) повторите 2 и 3 N количество раз, чтобы начали получаться целые слова
Параметры в gensim.models.Phrases влияют на количество получаемых нграммов после каждого прохода, поэтому не забудьте их настроить


In [40]:
symbol_sents = [' '.join(sent) for sent in sents]

In [41]:
symbol_sents = [[ch for ch in sent if ch not in ',.;!?\n'] for sent in symbol_sents]

In [61]:
phrases1 = Phrases(symbol_sents, scoring='npmi', threshold=1, min_count=3)

In [68]:
phrases2 = Phrases(phrases1[symbol_sents], scoring='npmi', threshold=0, min_count=2)

In [69]:
phrases3 = Phrases(phrases2[symbol_sents], scoring='npmi', threshold=-1)

In [70]:
list(phrases3[phrases2[phrases1[symbol_sents]]])

[['p_a_r',
  't_ _o_n',
  'e_ _H',
  'I_S',
  ' _M_A',
  'C_U_L_A',
  'T_E_ _O',
  'R_I_G',
  'I_N_ _I',
  ' _O_N_C',
  'E_ _U',
  'P_O_N',
  ' _A_ _T',
  'I_M_E',
  ' _w_h_e',
  'n_ _t_h',
  'e_ _w_o',
  'r_l_d',
  ' _w_a_s',
  ' _y_o_u',
  'n_g_ _t',
  'h_e_r_e',
  ' _w_a_s',
  ' _a_ _M',
  'a_r_t_i',
  'a_n_ ',
  'n_a_m',
  'e_d_ _S',
  'm_i_t_h'],
 ['V_a_l_e',
  'n_t_i_n',
  'e_ _M_i',
  'c_h_a',
  'e_l_ _S',
  'm_i_t_h',
  ' _w_a_s',
  ' _a_s_ ',
  'r_e_a_l',
  ' _a_s_ ',
  't_a_x',
  'e_s_ _b',
  'u_t_ _h',
  'e_ _w_a',
  's_ _a',
  ' _r',
  'a_c_e_ ',
  'o_f_ ',
  'o_n_e'],
 ['T_h_e_ ',
  'f_i_r_s',
  't_ _h',
  'u_m_a_n',
  ' _e_x',
  'p_e_d_i',
  't_i_o_n',
  ' _f_r_o',
  'm_ _T',
  'e_r_r',
  'a_ _t',
  'o_ _M',
  'a_r_s_ ',
  'w_a_s_ ',
  's_e_l_e',
  'c_t_e_d',
  ' _o_n',
  ' _t_h_e',
  ' _t_h_e',
  'o_r_y_ ',
  't_h_a_t',
  ' _t_h_e',
  ' _g_r_e',
  'a_t_e_s',
  't_ _d',
  'a_n_g_e',
  'r_ _t_o',
  ' _m_a_n',
  ' _i_n_ ',
  's_p_a_c',
  'e_ _w_a',
  's_ _m_a',
  'n_ _h_i',